# Entrega 5.2 - Jugador de Damas Chinas (Q Learning, Redes Neuronales)

### Grupo 07:
     - Renzo Gambone C.I. 5.155.486-4
     - Germán Ouviña C.I. 4.823.566-1
     - Leandro Rodríguez C.I 4.691.736-4


## 1. Introducción
***
### 1.1. Objetivo
***
A

### 1.2. Entrega
***
A

### 1.3. Formato
***
A

## 2. Diseño
***
A

### 2.1. Juego (*)
***
El juego **Damas Chinas** cuenta con múltiples variantes y reglas distintas. A continuación se especifican las reglas de su forma básica, así como los cambios implementados en este escenario con el motivo de simplificar u optimizar la resolución del problema:

**Jugadores:**
> - Puede haber entre 2 y 6 jugadores, los cuales juegan todos contra todos.
> - Cada jugador cuenta con 10 piezas.
> - *Simplificación:* Se limita la participación a 2 jugadores, con el objetivo de simplificar el problema a representar.

**Tablero:**
> - El tablero tiene forma de estrella de 6 puntas, el cual cuenta con 121 casillas dispuestas en forma hexagonal.
> - Esto implica que cada celda cuenta con 6 celdas adyacentes, a diferencia de una grilla rectangular, donde cada celda tiene 9 celdas adyacentes.
> - Cada jugador puede mover sus piezas hacia las celdas adyacentes que no estén ocupadas, o saltar piezas (suyas o del oponente) siempre y cuando el salto sea siguiendo la recta que contiene la pieza que salta y la pieza saltada.
> - Cada jugador comienza con sus 10 piezas en una punta de la estrella.
> - *Simplificación:* Se limita el tablero a ser un rombo de 81 casillas, eliminando los 4 triángulos que se encuentran a los lados del tablero. Dicha simplificación se realiza ya que en un juego entre 2, la existencia de dichos triángulos no tiene relevancia y solo agrega complejidad al representar el tablero.

<div>
    <div style="display: inline-block; width: 49%; text-align: center;">
        <img src="img/tableroOriginal.jpg" />
        <label style="margin-top: 16px; font-size: 16px; font-family: monospace;"> Figura 1 - Tablero original </label>
    </div>
    <div style="display: inline-block; width: 49%; text-align: center;">
        <img src="img/tableroSimple.jpg" />
        <label style="margin-top: 16px; font-size: 16px; font-family: monospace;"> Figura 2 - Tablero simplificado </label>
    </div>
</div>

**Objetivo:**
> - El objetivo de un jugador es depositar todas sus fichas en el triángulo opuesto a su triángulo de inicio.
> - El juego se termina cuando uno de los jugadores cumple dicho objetivo, no existiendo posibilidad de empate.
> - *Simplificación:* Debido a que es posible que un partido se extienda en el tiempo tanto como los jugadores lo deseen, se agrega la posibilidad de **empatar** una partida, declarandose dicho empate luego de pasada una cierta cantidad de turnos configurable (por defecto 100).

**Complejidad:**
> - A diferencia del Go y las fichas no coronadas de las damas ordinarias, en las damas chinas toda pieza puede moverse hacia atrás, adicionalmente un jugador podría realizar el movimiento inverso al movimiento previo, causando un bucle de movimientos que no alteran el estado final del campo.
> - *Simplificación:* Debido a la enorme cantidad de movimientos posibles (muchos de estos contraproducentes), se definió restringir movimientos atrás por parte de los jugadores, (salvo que no hayan posibles movimientos hacia adelante). También se decidió restringir movimientos inversos (que dejen las piezas del jugador en una dispocisión reciente al rederod del tablero) a los últimos tres movimientos por parte del jugador (salvo que no hayan más posibles movimientos).

En resumen, se realizan **5 modificaciones** a las reglas básicas del juego:
- Se limita la cantidad de jugadores a 2.
- Se transforma el tablero de una estrella de 121 casillas a un rombo de 81 casillas.
- Se agrega la posibilidad de empatar si no se logra terminar el juego luego de cierta cantidad de turnos.
- Sólo se permiten movimientos hacia atrás si el jugador no tiene más posibles movimientos hacia adelante.
- Se restringen movimientos inversos a los últimos tres realizados por el jugador (salvo que no queden más posibles movimientos). 

### 2.2. Tablero
***
Como ya se mencionó en la anterior sección, el tablero de **Damas Chinas** tiene una estructura particular. A continuación se trata el problema subdividiendo en varias secciones:

#### 2.2.1. Representación física y lógica (*)
***
Al tratarse de un tablero con una _grilla hexagonal_, no existe ninguna estructura predeterminada en Python para almacenarlo y mantener el movimiento de fichas sencillo e intuitivo. De esta forma, se generaron dos formas de representar el tablero: una **lógica** (para mostrarlo y realizar cálculos) y una **física** (para almacenarlo).

La **representación lógica** consta de un rombo de 81 casillas dispuestas hexagonalmente (cada casilla tiene 6 adyacentes excepeto las de los extremos) las cuales cuentan con un par de _coordenadas axiales_ para representarlas. Dicho sistema de coordenadas cuenta con 2 ejes como un sistema cartesiano, con la diferencia de que dichos ejes no son perpendiculares. Para obtener una referencia visual, correr **script 1** al final de la sección.

La **representación física**, por otra parte, consta de una matriz cuadrada 9x9 de 81 casillas dispuestas las cuales cuentan con un par de _coordenadas cartesianas_ para representarlas (una matriz común y corriente). Existe una _biyección_ entre las casillas en la representación lógica y en la física. Dicha biyección permite obtener las coordenadas axiales de un par de coordenadas cartesianas, existiendo una función inversa que permite hacer lo contrario. La biyección se compone de dos transformaciones lineales consecutivas:

1. **Traslación del centro:** Partiendo de la matriz, la cual cuenta con el centro en (0,0) en una esquina, se traslada dicho centro a la celda (4,4) (el centro exacto de la matriz cuadrada).
2. **Agregado de los extremos:** Partiendo de la matriz con centro trasladado, se sustituyen todas las coordenadas y que sean mayores a 4 o menores a -4 restandoles o sumandoles 9 respectivamente.

Sea $f: R^2 \rightarrow R^2$ la expresión que representa a la biyección, con $f_1$ y $f_2$ las anteriores transformaciones:

- $f(x,y)=f_2(f_1(x,y))$

- $f_1(x,y)=(x-4, -y+4)$

- $f_2(x,y)=$
    - $(x,y)$ $\forall$ $(x,y)$ $:$ $|y| < 4$
    - $(x,y-9)$ $\forall$ $(x,y)$ $:$ $y \geq 4$
    - $(x,y+9)$ $\forall$ $(x,y)$ $:$ $y \leq -4$

Para obtener una referencia visual, correr **script 1** al final de la sección.

#### 2.2.2. Representación de atributos
***
Se optó por trabajar con dos representaciones posibles para los atributos.
- **Métricas:** (Empleadas previamente en el laboratorio 1), estas mismas abstraen la representación del tablero a un nivel más alto que permitan acoplar tableros no necesariamente iguales en terminos de distribución en sus piezas pero sí en terminos de "calidad de la distribución de las mismas"
- **Tablero:** La dispocisión de piezas en el tablero, denotando para cada celda si está vacia, tiene una pieza del jugador 1 o tiene una pieza del jugador 2.

##### 2.2.2.1. Métricas (*)
***
Se determinaron **4 métricas**, las cuales deberían reflejar lineamientos generales sobre que tan bien está jugando el jugador _i_, estas son:
1. **$A_i$** - **Suma cuadrada de la distancia de cada pieza del jugador _i_ hacia el objetivo.** El objetivo de esta métrica es incitar al jugador _i_ a mover sus piezas al extremo opuesto del tablero.
2. **$B_i$** - **Suma cuadrada de la distancia de cada pieza del jugador _i_ hacia la línea vertical que representa el centro del tablero.** El objetivo de esta métrica es incitar al jugador _i_ a no alejar sus piezas de la línea central vertical del tablero, ya que esto en general consume movimientos y desperdicia posibles cadenas de saltos.
3. **$C_i$** - **Suma de la cantidad de saltos posibles de cada pieza del jugador _i_.** El objetivo de esta métrica es incitar al jugador _i_ a generar cadenas de saltos para hacer avanzar sus piezas más rápidamente.
4. **$D_i$** - **Suma cuadrada de la distancia de cada pieza del jugador _i_ hacia la casilla libre más cercana en el triángulo objetivo.** El objetivo de esta métrica es incitar al jugador _i_ a dirigirse al triángulo opuesto y rellenarlo con sus fichas. Se considera que esta métrica no sólo no es redundante con **$A_i$** , sino que además la complementa.

De esta forma, esta representación consta de un **Conjunto de 9 atributos** = $[1, A_1, B_1, C_1, D_1, A_2, B_2, C_2, D_2]$, el cual engloba las 8 métricas mencionadas anteriormente, agregando una para poder contar con un peso $w_0$ independiente en la función de evaluación.

Finalmente, es importante detallar que los atributos obtenidos se **normalizan** pasando a estar en el intervalo $[0..1]$. Dicha normalización se implementa utilizando la **norma euclídea** y siguiendo la siguiente fórmula, siendo $x$ el vector de atributos, $x_i$ la coordenada _i_-ésima del mismo, $z$ el vector de atributos normalizado y $z_i$ la coordenada _i_-ésima del mismo:

$$z : z_i = \frac{x_i}{||x||}$$  

$$\forall i \in [0..length(x)]$$

##### 2.2.2.2. Tablero
Como los modelos a entrenar para este laboratorio constan de una red neuroanl, se decidió considerar al tablero en sí como representación de los atributos.

De esta forma se considera al tablero como la matriz empleada en su representación, un rombo de dimensión 9x9 y este mismo se representa en un único vector de longitud 81, donde cada posición está vinculada a su respectiva celda en el tablero. Los posibles valores para cada celda son los siguientes:
- **1:** Si la celda es ocupada por una pieza del *Jugador 1*.
- **0:** Si la celda no es ocupada por ninguna pieza.
- **-1:** Si la celda es ocupada por una pieza del *Jugador 2*.

In [1]:
# A continuación se adjunta un script que permite comparar las distintas representaciones del tablero interactivamente, 
# extrayendo sus atributos y viendo como el mover piezas afecta a los mismos.
# Script 1 - Representación del tablero
import ntShowBoard

Box(children=(Label(value='Representación del tablero:', layout=Layout(margin='0px 16px 0px 0px')), ToggleButt…


                                (   O   )

                            (   O   )(   O   )

                        (   O   )(   O   )(   O   )

                    (   O   )(   O   )(   O   )(   O   )

                (       )(       )(       )(       )(       )

            (       )(       )(       )(       )(       )(       )

        (       )(       )(       )(       )(       )(       )(       )

    (       )(       )(       )(       )(       )(       )(       )(       )

(       )(       )(       )(       )(       )(       )(       )(       )(       )

    (       )(       )(       )(       )(       )(       )(       )(       )

        (       )(       )(       )(       )(       )(       )(       )

            (       )(       )(       )(       )(       )(       )

                (       )(       )(       )(       )(       )

                    (   O   )(   O   )(   O   )(   O   )

                        (   O   )(   O   )(   O   )

                            (   O   )( 

Box(children=(Box(children=(Label(value='Movimiento de jugador 1:', layout=Layout(margin='0px 16px 0px 0px')),…

### 2.3. Modelo
***

De manera similar al laboratorio 1, los modelos implementados trabajan con conceptos de *aprendizaje por refuerzos*, orientandose por decisiones similares a las realizadas en el pasado (partidas anteriores) en base a si estas mismas tuvieron buen resultado (fueron recompensadas) o un mal resultado (fueron penalizadas).

El tipo de *aprenndizaje por refuerzos* con el que se trabaja es *Q-Learning* **(detallar?)**, sin embargo se representa la función **Q** mediante una red neuronal (en lugar de una tabla).

Es de esta forma que se determinó experimentar con dos posibles modelos, ambos compuestos por una red neuronal implementada por la clase *MLPRegressor* de la biblioteca [SKLearn](https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPRegressor.html), la cual procesa una entrada dada y retorna un posible valor para **Q**.

La arquitectura MLP (Multi-layer Perceptron), es una de las más básicas y tradicionales en el área de las redes neuronales y consiste en conectar cada una de las neuronas de la capa **n** con cada una de las neuronas de la capa **n+1**, componiendose de esta forma por una gran cantidad de parámetros (pesos y biases)

La diferencia radical entre ambos modelos son los atributos que estos mismos reciben en su *capa de entrada* (métricas o el tablero), al ser esta la única gran diferencia, ambos modelos tienen una cierta cantidad de caracteristicas en común, estas mismas son:
- Se trabaja con la flag *warm_start* en *True*, esto permite tener un entrenamiento iterativo en la red (a diferencia de un único entrenamiento determinado por un enorme dataset)
- asd

#### 2.3.1. Métricas como entrada
***

Basandonos en nuestro trabajo previo, se optó porque los atributos de entrada para este modelo sean las métricas que dieron un mejor resultado para el laboratorio 1 (detalladas en la sección anterior).

De esta forma este modelo es compuesto por una red formada por 9 neuronas en su *capa de entrada*, en donde cada una recibe un vector de métricas y determina un valor para **Q** procesando las mismas.

#### 2.3.2. Tablero como entrada
***

Debido a que las redes neuronales tienen la particularidad de determinar en base a los ejemplos "que es lo que resaltar de los mismos", se decidió que los atributos de este modelo sean los que mejor reflejan la información en el tablero actual (el tablero en sí).

De esta forma este modelo es compuesto por una red formada por 81 neuronas en su *capa de entrada*, en donde cada una recibe una celda en el campo y determina un valor para **Q** procesando las mismas.

### 2.4. Algoritmo
***
A

#### 2.4.1. Algoritmo de entrenamiento
***
Aca hablar de lo que hace sklearn pa entrenar (sgd, back propagation)

#### 2.4.2. Algoritmo de clasificación
***
Acá hablar de lo que hace sklearn pa clasificar (forward coso)

#### 2.4.3. Arquitectura
***
A

##### 2.4.3.1. Arquitectura - Capa de entrada
***
Acá hablar de cantidad de neuronas (9 u 81)

##### 2.4.3.2. Arquitectura - Capas ocultas
***
Acá hablar de cantidad de hidden layers y cantidad de neuronas en cada una

##### 2.4.3.3. Arquitectura - Capa de salida
***
Acá hablar de cantidad de neuronas (1)

#### 2.4.4. Parámetros
***
A

##### 2.4.4.1. Parámetros - Función de activación
***
Acá hablar de ReLU (y sigmoide y tanh, veremos si nos da pa probar)

##### 2.4.4.2. Parámetros -  Pesos iniciales
***
Acá hablar de que MLP genera pesos random (casi seguro)

##### 2.4.4.3. Parámetros -  Ratio de aprendizaje y enfriamiento
***
Acá hablar de ratios constantes y variables y sus valores


### 2.5. Oponentes
***
A

#### 2.5.1. Jugador aleatorio (*)
***
A

#### 2.5.2. Jugador previo (Red neuronal)
***
A

#### 2.5.3. Jugador entrenado (Función lineal)
***
A

### 2.6. Evaluación
***
A

#### 2.6.1. Métodos
***
A

#### 2.6.2. Métricas
***
A

## 3. Experimentación
***
A

### 3.1. Metodología
***
A

1. **Configuraciones paramétricas:** Entrenar muchas redes distintas y ver como juega cada una. Aplicar las mismas configs pa self y pa random.
<br><br>

2. **Elección y comparación de representantes:** Analisis cuantitativo (comparar winRate y victoryRate entre redes) y cualitativo (ver como se distribuyen las victorias y el ECM en el tiempo para los mejores). Elegir las mejores 8 self y los mejores 8 random.
<br><br>

3. **Ponderación de representantes:** Torneoooo entre las mejores 8 anteriores, formato partido de fubol. 3 ganar, 1 empatar, 0 perder. Luego de los 64 partidos se ve quien tuvo mejor puntaje en la tabla. Si hay empates, se hace eliminatoria con los empatantes. Un torneo pa self y uno pa random.
<br><br>

4. **Comparación con jugador lineal:** El ganador del torneo VS el mejor del lab1
<br><br>


### 3.2. Configuraciones paramétricas
***

#### 3.2.1. Instancias
***
A

#### 3.2.2. Parámetros
***
A

#### 3.2.3. Resultados
***
A

In [ ]:
# Script que permite entrenar una red con cierta config y ver sus resultados

In [ ]:
# Script que grafica y muestra tabla sobre resultados de cada configuración paramétrica entrenada por nosotros

### 3.3. Elección y comparación de representantes
***
A

#### 3.3.1. Análisis cuantitativo
***
A

#### 3.3.2. Análisis cualitativo
***
A

#### 3.3.3. Representantes
***
A

**Observaciones generales:**
- A

**Interpretaciones:**
- A


### 3.4. Ponderación de representantes
***
A

In [ ]:
# Script que permite simular torneo

In [ ]:
# Script que grafica y muestra tabla de resultados de torneo

### 3.5. Comparación con jugador lineal
***
A

In [ ]:
# Script que permite simular partido entre la mejor red y el lineal

In [ ]:
# Script que grafica y muestra tabla de resultados del partido

## 4. Conclusiones
***

#### 4.1. Respecto al Modelo
***
A

#### 4.2. Respecto a los Parámetros
***
A

#### 4.3. Respecto a los Resultados
***
A

#### 4.4. Respecto a los Oponentes
***
A

#### 4.5. Posibles mejoras
***
A:
- A
- A